In [1]:
import socket
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from os import urandom
from cryptography.fernet import Fernet

# Упражнение 1

1. Расширение функционала клиента.

* Модифицируйте код клиентского приложения так, чтобы он мог читать сообщения, введенные пользователем в консоли, и отправлять их серверу. После отправки каждого сообщения клиент должен получать и выводить эхо-ответ от сервера.

* Реализуйте возможность для пользователя завершить работу клиентского приложения по специальной команде, например, введя `exit`. Это должно корректно закрывать соединение с сервером и останавливать программу клиента.

2. Обработка исключений.

* Добавьте в код сервера обработку исключений для ситуаций, когда клиент неожиданно разрывает соединение. Убедитесь, что сервер продолжает работу и готов принимать новые подключения даже после возникновения ошибок.

* Реализуйте обработку исключений в клиентском приложении для обработки ошибок сети, таких как невозможность подключиться к серверу. В случае ошибки должно выводиться соответствующее сообщение, и клиент должен корректно завершать работу.

3. Расширение функционала сервера.

* Дополните сервер функционалом логирования. Сервер должен записывать в лог-файл информацию о каждом подключении: время подключения и адрес клиента. Также сервер должен логировать сообщения, получаемые от клиентов.

* Разработайте механизм обработки специальных команд от клиента, например, `shutdown`, который будет корректно завершать работу сервера. Убедитесь, что сервер закрывает все активные соединения перед выключением

In [5]:
def start_client():
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 9090))
        
        while True:
            data = input()
            client_socket.sendall(data.encode())
            response = client_socket.recv(1024)
            
            if response.decode('utf-8') == 'exit':
                print('Клиентский сокет закрыт')
                break
                
            if response.decode('utf-8') == 'shutdown':
                print('Сервер и клиентский сокет закрыты')
                break
                
            print(f"Ответ от сервера: {response.decode('utf-8')}")
            
    except ConnectionRefusedError:
        print('Невозможно подключиться к серверу')

    finally:
        client_socket.close()

In [9]:
start_client()

vdvddv
Ответ от сервера: vdvddv
bdbfbed
Ответ от сервера: bdbfbed
exit
Клиентский сокет закрыт


# Упражнение 3. Реализация протокола Диффи-Хеллмана и асимметричное шифрование

In [2]:
def start_client():
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 9090))
        
        params_bytes = client_socket.recv(1024)
        parameters = serialization.load_pem_parameters(params_bytes, backend=default_backend())
        
        private_key = parameters.generate_private_key()
        public_key = private_key.public_key()
        
        public_key_b = public_key.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo
        )
        client_socket.sendall(public_key_b)
        
        server_public_key_b = client_socket.recv(1024)
        server_public_key = serialization.load_pem_public_key(server_public_key_b)
        
        shared_secret = private_key.exchange(server_public_key)
        derived_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info='handshake data'.encode(),
            backend=default_backend()
        ).derive(shared_secret)
        
        while True:
            data = input().encode('utf-8')
            iv = urandom(16)
            cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend=default_backend())
            encryptor = cipher.encryptor()
            ciphertext = encryptor.update(data) + encryptor.finalize()
            
            client_socket.sendall(iv)
            client_socket.sendall(ciphertext)
            
            iv = client_socket.recv(1024)
            cipher_response = client_socket.recv(1024)

            cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend=default_backend())
            decryptor = cipher.decryptor()
            response = decryptor.update(cipher_response) + decryptor.finalize()
            
            if response.decode('utf-8') == 'exit':
                print('Клиентский сокет закрыт')
                break
                
            if response.decode('utf-8') == 'shutdown':
                print('Сервер и клиентский сокет закрыты')
                break
                
            print(f"Ответ от сервера: {response.decode('utf-8')}")
            
    except ConnectionRefusedError:
        print('Невозможно подключиться к серверу')

    finally:
        client_socket.close()

In [6]:
start_client()

okkookok
Ответ от сервера: okkookok
shutdown
Сервер и клиентский сокет закрыты


# Упражнение 5. Работа с UDP-сокетами

In [2]:
def start_udp_client():
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        server_address = ('localhost', 9090)
        
        while True:
            data = input()
            client_socket.sendto(data.encode(), server_address)
            response, _ = client_socket.recvfrom(1024)
                
            if response.decode('utf-8') == 'exit':
                print('Сервер и клиентский сокет закрыты')
                break
                
            print(f"Ответ от сервера: {response.decode()}")
            
    except ConnectionRefusedError:
        print('Невозможно подключиться к серверу')

    finally:
        client_socket.close()

In [5]:
start_udp_client()

ghjkl
Ответ от сервера: ghjkl
okj
Ответ от сервера: okj
vdvj
Ответ от сервера: vdvj
exit
Сервер и клиентский сокет закрыты


# Упражнение 6. Использование селекторов для обработки множественных соединений

In [9]:
start_client()

rvrvr
Ответ от сервера: rvrvr
shutdown
Сервер и клиентский сокет закрыты


# Лабораторная работа №8

In [2]:
def start_client():
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 9090))
        
        params_bytes = client_socket.recv(1024)
        parameters = serialization.load_pem_parameters(params_bytes, backend=default_backend())
        
        private_key = parameters.generate_private_key()
        public_key = private_key.public_key()
        
        public_key_b = public_key.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo
        )
        client_socket.sendall(public_key_b)
        
        server_public_key_b = client_socket.recv(1024)
        server_public_key = serialization.load_pem_public_key(server_public_key_b)
        
        shared_secret = private_key.exchange(server_public_key)
        derived_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info='handshake data'.encode(),
            backend=default_backend()
        ).derive(shared_secret)
        
        while True:
            data = input().encode('utf-8')
            iv = urandom(16)
            cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend=default_backend())
            encryptor = cipher.encryptor()
            ciphertext = encryptor.update(data) + encryptor.finalize()
            
            client_socket.sendall(iv)
            client_socket.sendall(ciphertext)
            
            iv = client_socket.recv(1024)
            cipher_response = client_socket.recv(1024)

            cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend=default_backend())
            decryptor = cipher.decryptor()
            response = decryptor.update(cipher_response) + decryptor.finalize()
            
            if response.decode('utf-8') == 'exit':
                print('Клиентский сокет закрыт')
                break
                
            if response.decode('utf-8') == 'shutdown':
                print('Сервер и клиентский сокет закрыты')
                break
                
            print(f"Ответ от сервера: {response.decode('utf-8')}")
            
    except ConnectionRefusedError:
        print('Невозможно подключиться к серверу')

    finally:
        client_socket.close()

In [4]:
start_client()

iji
Ответ от сервера: iji
rfrf
Ответ от сервера: rfrf


KeyboardInterrupt: Interrupted by user

In [7]:
def start_udp_client():
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        server_address = ('localhost', 9090)
        
        client_socket.sendto('Получение ключа'.encode(), server_address)
        key, _ = client_socket.recvfrom(1024)
        cipher = Fernet(key)
        while True:
            data = input().encode('utf-8')
            ciphertext = cipher.encrypt(data)
            client_socket.sendto(ciphertext, server_address)
 
            cipher_response, _ = client_socket.recvfrom(1024)
            response = cipher.decrypt(cipher_response)
                
            if response.decode('utf-8') == 'exit':
                print('Сервер и клиентский сокет закрыты')
                break
                
            print(f"Ответ от сервера: {response.decode()}")
            
    except ConnectionRefusedError:
        print('Невозможно подключиться к серверу')

    finally:
        client_socket.close()

In [8]:
start_udp_client()

ыдаодыоа
Ответ от сервера: ыдаодыоа
ыдаыдола
Ответ от сервера: ыдаыдола
exit
Сервер и клиентский сокет закрыты


In [7]:
def start_client():
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 9090))
        
        key = client_socket.recv(1024)
        cipher = Fernet(key)
        while True:
            data = input().encode()
            ciphertext = cipher.encrypt(data)
            client_socket.sendall(ciphertext)
            
            cipher_response = client_socket.recv(1024)
            response = cipher.decrypt(cipher_response)
            
            if response.decode('utf-8') == 'exit':
                print('Клиентский сокет закрыт')
                break
                
            if response.decode('utf-8') == 'shutdown':
                print('Сервер и клиентский сокет закрыты')
                break
                
            print(f"Ответ от сервера: {response.decode('utf-8')}")
            
    except ConnectionRefusedError:
        print('Невозможно подключиться к серверу')

    finally:
        client_socket.close()

In [8]:
start_client()

jsjsj
Ответ от сервера: jsjsj
jcsjcjsic
Ответ от сервера: jcsjcjsic
jdjnjdnvndv
Ответ от сервера: jdjnjdnvndv
shutdown
Сервер и клиентский сокет закрыты
